# Setup

In [29]:
# !pip install -qU openai
# from google.colab import userdata

In [30]:
from openai import OpenAI
import time

In [31]:
class CFG:
    base_url = "http://localhost:11434/v1"
    api_key = "ollama"  # required, but unused
    model = "gpt-4o-mini"
    num_tokens = 2000
    temperature = 0.7

Ta część kodu definiuje klasę konfiguracyjną `CFG`, która służy do przechowywania ustawień modelu językowego. Jest to przykład zastosowania wzorca programistycznego znanego jako klasa konfiguracyjna - centralne miejsce do zarządzania parametrami aplikacji.

W tym konkretnym przypadku klasa zawiera tylko jedną zmienną klasową `model`, której przypisano wartość "gpt-4o-mini". Zmienna ta określa, który model OpenAI będzie używany do przetwarzania zapytań. Jest to szczególnie istotne, ponieważ różne modele OpenAI mają różne możliwości, koszty i limity.

Używanie klasy konfiguracyjnej zamiast zwykłej zmiennej ma kilka zalet. Po pierwsze, wszystkie ustawienia są zgrupowane w jednym miejscu, co ułatwia ich zarządzanie i modyfikację. Po drugie, taka struktura pozwala na łatwe dodawanie kolejnych parametrów w przyszłości bez konieczności modyfikacji pozostałych części kodu. Po trzecie, klasa może być importowana i używana w różnych miejscach projektu, co zapewnia spójność konfiguracji w całej aplikacji.

Jest to podobne do tablicy rozdzielczej w samochodzie - wszystkie ważne wskaźniki i ustawienia są zgrupowane w jednym miejscu, dzięki czemu kierowca ma do nich łatwy dostęp i może szybko sprawdzić lub zmienić potrzebne parametry.

In [32]:
from dotenv import load_dotenv
import os

load_dotenv()

def assert_env_key(key: str | None, name: str) -> None:
    if not key:
        raise ValueError(f"{name} is not set, please update .env file")
    
OPENAI_API_KEY_NAME = "OPENAI_API_KEY"
OPENAI_API_KEY = os.getenv(OPENAI_API_KEY_NAME)
# OPENAI_API_KEY = userdata.get(OPENAI_API_KEY_NAME) # Google colab env retrieval option
assert_env_key(OPENAI_API_KEY, OPENAI_API_KEY_NAME)

client = OpenAI(api_key=OPENAI_API_KEY)
# client = OpenAI(api_key = userdata.get('openaivision'))

# Eksperymant

## Tworzenie nowego asystenta w OpenAI API

Ten kod tworzy asystenta w OpenAI API z określonymi ustawieniami:  

---

##### 🔹 **Metoda tworzenia:**
- **`client.beta.assistants.create()`** – Używana do utworzenia asystenta przez API w wersji beta.  

---

#### 🔹 **Parametry:**
- **`name = "helper2"`** – Nazwa asystenta to "helper2". Ułatwia identyfikację.  

- **`instructions`** – Wytyczne określające rolę i zachowanie asystenta jako **"Business AI Guide"**:  
  - Upraszcza techniczne informacje na język biznesowy  
  - Unika żargonu technicznego  
  - Zachowuje formalny ton  
  - Jest informacyjny i cierpliwy  
  - Unika szczegółów technicznych, chyba że wymagane  
  - Nie angażuje się w dyskusje polityczne  
  - Nie spekuluje  
  - **Odpowiada "I do not know"**, gdy nie jest pewny odpowiedzi  
  - Prosi o wyjaśnienie **tylko raz na zapytanie**  

- **`model = CFG.model`** – Określa model asystenta na podstawie konfiguracji w klasie `CFG`.  

---

#### 🔹 **Kontynuacja linii:**
- **`\`** na końcu linii pozwala na kontynuację długiego tekstu w kolejnych liniach, poprawiając czytelność kodu.  


In [33]:
assistant = client.beta.assistants.create(
    name = "helper2",
    instructions = "Your role as 'Business AI Guide' is to assist users with questions \
                    and explanations about Artificial Intelligence (AI) and \
                    Machine Learning (ML), tailored for business professionals. \
                    You should simplify complex technical information into clear,\
                    business-friendly language, avoiding technical jargon. \
                    Maintain a formal tone in your communications, being \
                    informative and patient, ensuring clarity for users\
                    without a technical background. Avoid detailed \
                    technical explanations unless specifically requested. \
                    Refrain from discussing politics or current affairs, \
                    and avoid speculating. If uncertain about an answer,\
                    respond with 'I do not know.' Limit yourself to asking \
                    for clarification only once per query; if the query \
                    remains unclear after that, provide the best answer \
                    you can, filling in any missing details as needed.",
     model = CFG.model,
)

## Tworzenie nowego wątku konwersacji w OpenAI API

Ten kod tworzy nowy wątek konwersacji i wyświetla jego szczegóły:  

---

### 🔹 **Tworzenie wątku:**
- **`thread = client.beta.threads.create()`** – Tworzy nowy wątek konwersacji.  
  - **Wątek** działa jak kontener dla wymiany wiadomości między użytkownikiem a asystentem.  
  - Zapewnia **ciągłość i kontekst rozmowy**.  
  - Po utworzeniu wątku API generuje unikalny identyfikator oraz inne metadane.  
  - To jak zakładanie nowego zeszytu – każda wiadomość będzie przypisana do tego samego wątku, co pozwala zachować **spójność konwersacji**.  

---

### 🔹 **Wyświetlanie szczegółów:**
- **`print(thread)`** – Wyświetla szczegóły nowo utworzonego wątku:  
  - Unikalny **identyfikator wątku**  
  - **Czas utworzenia**  
  - Inne **metadane** związane z wątkiem  
  - Pomaga to w **debugowaniu** oraz weryfikacji poprawności utworzenia wątku.  

---

### 🔹 **Dlaczego to ważne?**
- Mechanizm wątków umożliwia prowadzenie **wielu równoległych konwersacji**, każdej z **własnym kontekstem i historią**.  
- To jak prowadzenie kilku rozmów w osobnych pokojach – każda ma **swój przebieg i kontekst**, nie mieszając się z innymi.  


In [34]:
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_u1zjmqIbjGfkVRDB1FY0MfYL', created_at=1739784403, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


## Tworzenie wiadomości w wątku konwersacji w OpenAI API

Te linie kodu tworzą nową wiadomość w utworzonym wcześniej wątku konwersacji, wykorzystując API OpenAI:  

---

### 🔹 **Metoda dodawania wiadomości:**
- **`client.beta.threads.messages.create()`** – Metoda, która dodaje wiadomość do istniejącego wątku.  

---

### 🔹 **Parametry:**
- **`thread_id = thread.id`** – Określa, do którego wątku ma zostać dodana wiadomość. Używamy identyfikatora wątku z poprzedniego kroku. To jak wskazanie zeszytu, do którego chcemy dodać notatkę.  

- **`role = "user"`** – Określa, że wiadomość pochodzi od użytkownika. W OpenAI rozróżnianie ról ma kluczowe znaczenie, ponieważ wpływa na sposób interpretacji wiadomości przez asystenta. Rola działa jak etykieta, która mówi, kto wysłał wiadomość.  

- **`content = "What is generative AI?"`** – Treść wiadomości. W tym przypadku pytanie o sztuczną inteligencję generatywną, na które asystent udzieli odpowiedzi.  

---

### 🔹 **Jak to działa?**
System wiadomości działa jak zapisywanie w zeszycie – każda wiadomość to nowy wpis z informacjami o **roli** nadawcy i **treści**.  
Wszystkie wiadomości są połączone poprzez identyfikator wątku, co pozwala asystentowi utrzymać **ciągłość rozmowy i kontekst**.


In [35]:
message = client.beta.threads.messages.create(
    thread_id = thread.id,
    role = "user",
    content = "What is generative AI?"
)

## Inicjowanie wykonania wątku konwersacji w OpenAI API

Ten fragment kodu inicjuje wykonanie (run) wątku konwersacji, łącząc go z wcześniej utworzonym asystentem. To moment, w którym system zaczyna przetwarzać wiadomość i generować odpowiedź.  

---

### 🔹 **Metoda inicjowania wątku:**
- **`client.beta.threads.runs.create()`** – Inicjuje wykonanie wątku konwersacji i łączy go z asystentem.  

---

### 🔹 **Parametry:**
- **`thread_id = thread.id`** – Określa, który wątek konwersacji ma zostać "uruchomiony". To jak przekazanie zeszytu z pytaniem do nauczyciela – musimy wskazać, którą rozmowę asystent ma przeanalizować i na co odpowiedzieć.  

- **`assistant_id = assistant.id`** – Określa, który asystent ma zająć się tym wątkiem. Łączymy wątek z asystentem, który posiada odpowiednie instrukcje i specjalizację (np. asystent biznesowy).  

---

### 🔹 **Jak to działa?**
- **System OpenAI** analizuje historię wątku, bierze pod uwagę instrukcje asystenta i rozpoczyna proces generowania odpowiedzi.  
- To jak moment, gdy nauczyciel czyta pytanie ucznia i zastanawia się nad odpowiedzią, uwzględniając wcześniejszą dyskusję i swoje wytyczne.  

---

### 🔹 **Ważna uwaga:**
- Utworzenie "run" nie oznacza natychmiastowej odpowiedzi – to **inicjowanie procesu** jej tworzenia.  
- To jak przekazanie pytania nauczycielowi – odpowiedź nadejdzie, ale potrzebny jest czas na jej przygotowanie.  


In [36]:
run = client.beta.threads.runs.create(
    thread_id = thread.id,
    assistant_id= assistant.id
)

## Pętla oczekiwania na odpowiedź od asystenta w OpenAI API

Ta pętla **`while True`** implementuje mechanizm oczekiwania i pobierania odpowiedzi od asystenta. Oto jak działa krok po kroku:

---

### 🔹 **Sprawdzanie statusu wątku:**
- **`client.beta.threads.runs.retrieve()`** – Sprawdza status wykonania wątku, używając identyfikatorów wątku (`thread_id`) i wykonania (`run_id`).  
  - To jak zaglądanie nauczycielowi przez ramię, aby sprawdzić, czy odpowiedź jest gotowa.  

---

### 🔹 **Przerwa między sprawdzeniami:**
- **`time.sleep(10)`** – Wprowadza 10-sekundową przerwę między kolejnymi zapytaniami do API.  
  - Ważne, aby nie przeciążać API zbyt częstymi zapytaniami i dać systemowi czas na przetworzenie odpowiedzi.  

---

### 🔹 **Sprawdzanie zakończenia procesu:**
- **`if run_status.status == 'completed'`** – Sprawdza, czy asystent zakończył już pracę nad odpowiedzią.  
  - **`messages = client.beta.threads.messages.list(thread_id=thread.id)`** – Pobiera historię wiadomości z wątku, w tym odpowiedź asystenta.  
  - **`break`** – Kończy pętlę, gdy odpowiedź jest gotowa.

---

### 🔹 **Dodatkowe opóźnienie:**
- **`else` z `time.sleep(2)`** – Jeśli odpowiedź nie jest jeszcze gotowa, dodaje 2-sekundową przerwę.  
  - Pomaga to w zarządzaniu zasobami i ogranicza częstotliwość zapytań.  

---

### 🔹 **Jak to działa?**
Cały ten mechanizm przypomina czekanie na odpowiedź od eksperta: regularnie sprawdzamy, czy skończył, ale robimy to w rozsądnych odstępach czasu, nie zaglądając co sekundę. Gdy odpowiedź jest gotowa, możemy ją odczytać i przetwarzać.  

---

### 🔹 **Przydatność:**
- Tego typu mechanizm jest szczególnie użyteczny w **systemach asynchronicznych**, gdzie czas generowania odpowiedzi może być zmienny, zależny od złożoności pytania i obciążenia systemu.  


In [37]:
while True:
    run_status = client.beta.threads.runs.retrieve(thread_id=thread.id,run_id=run.id)
    if run_status.status == 'completed':
        messages = client.beta.threads.messages.list(thread_id=thread.id)
        break
    elif run_status.status == 'failed':
        print(f"Error details (if available): {run_status.last_error.message}")
        raise
    else:
        time.sleep(10)

Error details (if available): You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.


RuntimeError: No active exception to reraise

In [ ]:
for message in reversed(messages.data):
    print(message.role + ":" + message.content[0].text.value)

user:What is generative AI?
